<a href="https://colab.research.google.com/github/Pankaj-2003/Strategy_Performance_Python/blob/main/Quant_Analyst_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime , timedelta

## Dataframe columns
### Date (datetime64[ns]): The date and time of the trade
### Symbol (string): The ticker symbol of the traded security
### Side (string): Either 'buy' or 'sell'
### Size (float, optional): The number of shares traded (default to 1 if not provided)
### Price (float): The price at which the trade was executed

In [2]:
def getTickerPrice(ticker: str, date: datetime) -> float:
    # This function returns the price of the security 'ticker' at the given 'date'
    # For the purpose of this exercise, assume it returns a random number
    return random.uniform(1, 100)  # Example implementation

In [ ]:
metrics = ["Sharpe" , "max-drawdown" ,  "profit factor" , "total net profit" , "Percent Profitable" , "average win\loss Ratio" , "Expectancy" , "Recovery factor" , "average r multiple" , "ROI"]

In [3]:
def calculate_sharpe(daily_returns, risk_free_rate=0.01):
    average_daily_return = daily_returns.mean()
    std_daily_return = daily_returns.std()

    # Avoid division by zero
    if std_daily_return == 0:
        return np.inf

    sharpe_ratio = (average_daily_return - risk_free_rate) / std_daily_return
    return sharpe_ratio

In [4]:
def calculate_max_drawdown(daily_returns):
    cumulative_return = (1 + daily_returns).cumprod()
    cumulative_max = cumulative_return.cummax()
    drawdown = cumulative_return - cumulative_max
    max_drawdown = drawdown.min()

    return max_drawdown

In [5]:
def calculate_profit_factor(trades):
    winning_trades = trades[trades['Return'] > 0]
    losing_trades = trades[trades['Return'] < 0]

    total_profit = winning_trades['Return'].sum()
    total_loss = -losing_trades['Return'].sum()  # Negate to get positive loss value

    if total_loss == 0:
        return np.inf if total_profit > 0 else 0  # Handle edge cases

    profit_factor = total_profit / total_loss
    return profit_factor


In [6]:
def percent_profitable(trades):
  winning_trades = trades[trades["Return"] > 0]
  total_trades = len(trades)
  percent_profitable = (len(winning_trades) / total_trades) * 100 if total_trades > 0 else 0
  return percent_profitable

In [7]:
def average_win_loss_ratio(trades):
    average_win = trades[trades["Return"] > 0]["Return"].mean() if len(trades[trades["Return"] > 0]) > 0 else 0
    average_loss = trades[trades["Return"] < 0]["Return"].mean() if len(trades[trades["Return"] < 0]) > 0 else 0

    average_win_loss_ratio = average_win / average_loss if average_loss > 0 else 0
    return average_win_loss_ratio


In [8]:
def expectancy(trades):
  average_profit = trades[trades["Return"] > 0]["Return"].mean() if len(trades[trades["Return"] > 0]) > 0 else 0
  average_loss = trades[trades["Return"] < 0]["Return"].mean() if len(trades[trades["Return"] < 0]) > 0 else 0
  winning_trades = trades[trades["Return"] > 0]
  losing_trades = trades[trades["Return"] < 0]
  total_trades = len(trades)
  win_rate = len(winning_trades) / total_trades if total_trades > 0 else 0
  loss_rate = len(losing_trades) / total_trades if total_trades > 0 else 0

  expectancy = (win_rate * average_profit) - (loss_rate * average_loss)
  return expectancy

In [9]:
def recovery_factor(trades  ,daily_returns):
  total_profit = trades[trades['Return'] > 0]['Return'].sum()
  total_loss = -trades[trades['Return'] < 0]['Return'].sum()  # Negate to get positive loss value
  total_net_profit = total_profit - total_loss
  max_dd = calculate_max_drawdown(daily_returns)
  recovery_factor = total_net_profit / abs(max_dd) if max_dd != 0 else np.inf
  return recovery_factor

In [49]:
def trade_perf(trades : pd.DataFrame) -> pd.Series:
  # if dataframe is empty
  if trades.empty:
    return pd.Series({
            'Sharpe': 0,
            'max-drawdown': 0,
            'profit factor': 0,
            'total net profit': 0,
            'Percent Profitable': 0,
            'average win\loss Ratio': 0,
            'Expectancy': 0,
            'Recovery factor': 0,
            'volatility': 0,
            'ROI': 0})

  # handling missing values

  trades['Size'] = trades['Size'].fillna(1)
  trades = trades[trades['Symbol'].notna()]
  trades = trades[trades['Side'].notna()]

  # seperate way to fill price
  def fill_missing_prices(row):
    if pd.isna(row['Price']):
        row['Price'] = getTickerPrice(row['Symbol'], row['Date'])
    return row
  trades = trades.apply(fill_missing_prices, axis=1)




  # calculating returns
  def calculate_return(row):
    current_price = getTickerPrice(row['Symbol'], row['Date'])
    if row['Side'] == 'buy':
        return (current_price - row['Price']) * row['Size'] , current_price
    else:
        return (row['Price'] - current_price) * row['Size'] , current_price

  trades['Return'] = trades.apply(calculate_return, axis=1 )  # Applying the function to each row of the df
  trades["curent_price"] = trades.apply(calculate_return , axis = 1 , result_type='expand')
  daily_returns = trades.groupby('Date')['Return'].sum() # daily return for sharpe
  print(daily_returns)
  return trades


  # # 1- calculating metrics

  # sharpe_ratio = calculate_sharpe(daily_returns)

  # # 2 - max_drawdown
  # maxdd = calculate_max_drawdown(daily_returns)

  # # 3 - profit factor
  # profit_factor = calculate_profit_factor(trades)

  # # 4 - total net profit
  # total_profit = trades[trades['Return'] > 0]['Return'].sum()
  # total_loss = -trades[trades['Return'] < 0]['Return'].sum()  # Negate to get positive loss value
  # total_net_profit = total_profit - total_loss

  # # 5 - percent profitable
  # perc_profitable = percent_profitable(trades)

  # # 6 - average win/loss ratio
  # avg_win_loss_ratio = average_win_loss_ratio(trades)

  # # 7 - expectancy
  # expectncy = expectancy(trades)

  # # 8 - recovery factor
  # recovery_factr = recovery_factor(trades  ,daily_returns)

  # # 9 - volatility
  # volatility = trades['Return'].std()

  # # 10 - roi
  # total_investement = (trades["Price"]* trades["Size"]).sum()
  # roi = (total_net_profit / total_investement) * 100 if total_investement != 0 else 0


  # metrics = pd.Series({
  #           'Sharpe': sharpe_ratio,
  #           'max-drawdown': maxdd,
  #           'profit factor': profit_factor,
  #           'total net profit': total_net_profit,
  #           'Percent Profitable': percent_profitable,
  #           'average win\loss Ratio': average_win_loss_ratio,
  #           'Expectancy': expectancy,
  #           'Recovery factor': recovery_factor,
  #           'volatility': volatility,
  #           'ROI': roi})

  # return metrics

In [51]:
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']
sides = ['buy', 'sell']
base_date = datetime(2024, 7, 15)
dates = [base_date + timedelta(days=i//4) for i in range(20)]
data = {
    'Date': dates,
    'Symbol': [random.choice(symbols) for _ in range(20)],
    'Side': [random.choice(sides) for _ in range(20)],
    'Size': [random.randint(1, 100) for _ in range(20)],
    'Price': [random.uniform(50, 300) for _ in range(20)],
}
trades = pd.DataFrame(data)




In [52]:
# Introduce missing values randomly
for col in ['Symbol', 'Size', 'Price']:
    trades.loc[random.sample(range(len(trades)), k=5), col] = np.nan  # Set 5 random entries in each column to NaN

In [ ]:
daily_returns = trades.groupby('Date')['Return'].sum()

In [47]:
trades

,Date,Symbol,Side,Size,Price
0,2024-07-15,GOOGL,buy,53.0,NaN
1,2024-07-15,AAPL,buy,14.0,211.072949
2,2024-07-15,MSFT,buy,38.0,233.347551
3,2024-07-15,NaN,buy,NaN,148.881432
4,2024-07-16,NaN,sell,33.0,NaN
5,2024-07-16,NaN,buy,25.0,NaN
6,2024-07-16,NaN,buy,NaN,113.815603
7,2024-07-16,AAPL,buy,31.0,275.887815
8,2024-07-17,AMZN,sell,65.0,254.699306
9,2024-07-17,TSLA,buy,NaN,213.085102


In [53]:
trade_perf(trades)

ValueError: Cannot set a DataFrame with multiple columns to the single column curent_price